## graphe generale 

In [2]:
import pandas as pd
from dash import Dash, dash_table, dcc, html, Input, Output
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx


# Charger les fichiers CSV
df = pd.read_csv("gapminder_unfiltered.csv")
linkedin_df = pd.read_csv(r"C:\Users\dell\OneDrive\Desktop\cmc\2eme\visuilisation yassin\linkedin_data.csv")

# Créer une base pour le Scatter 3D
data = {
    "age": [23, 45, 31, 35, 40, 29, 22, 34],
    "revenue": [3000, 7000, 5000, 8000, 9000, 4500, 3200, 5600],
    "annees d'etudes": [12, 18, 15, 17, 20, 14, 12, 16],
}
scatter_df = pd.DataFrame(data)

# Préparer les données pour le graphe réseau
NewData = linkedin_df.groupby('Company')[['First Name', 'Last Name']].size().reset_index(name='Count')
NewData = NewData.sort_values(by="Count", ascending=False).head(25)
my_name = "Anastasia Gorina"
NewData['My Name'] = my_name

# Initialiser l'application Dash
app = Dash(__name__)

# Définir la mise en page
app.layout = html.Div(
    style={"padding": "20px", "font-family": "Arial"},
    children=[
        html.H1(
            'Application Interactif : Tableaux, Graphiques et Réseaux',
            style={
                'color': 'black',
                'textAlign': 'center',
                'background-color': 'lightgray',
                'padding': '10px',
                'border-radius': '10px',
            },
        ),
        html.Div(
            style={"display": "flex", "gap": "20px", "flex-wrap": "wrap"},
            children=[
                # Tableau interactif
                html.Div(
                    style={"flex": "1", "min-width": "400px"},
                    children=[
                        html.H3("Tableau et Filtrage"),
                        dcc.Input(
                            id="filter-input",
                            type="text",
                            placeholder="Rechercher (par pays, continent...)",
                            debounce=True,
                            style={"width": "100%", "padding": "10px"},
                        ),
                        dash_table.DataTable(
                            id="data-table",
                            columns=[{"name": col, "id": col} for col in df.columns],
                            data=df.to_dict("records"),
                            page_size=10,
                            style_table={"overflowX": "auto"},
                        ),
                    ],
                ),
                # Graphique interactif
                html.Div(
                    style={"flex": "1", "min-width": "400px"},
                    children=[
                        html.H3("Graphique interactif"),
                        dcc.Dropdown(
                            id="country-dropdown",
                            options=[
                                {"label": country, "value": country}
                                for country in df["country"].unique()
                            ],
                            value="Canada",
                        ),
                        dcc.Graph(id="line-chart"),
                    ],
                ),
            ],
        ),
        html.Div(
            style={"display": "flex", "gap": "20px", "margin-top": "20px"},
            children=[
                # Scatter 3D
                html.Div(
                    style={"flex": "1", "min-width": "400px"},
                    children=[
                        html.H3("Scatter Plot 3D"),
                        dcc.Graph(
                            id="scatter-3d",
                            figure=px.scatter_3d(
                                scatter_df,
                                x="age",
                                y="annees d'etudes",
                                z="revenue",
                                color="revenue",
                                title="3D Scatter Plot",
                            ),
                        ),
                    ],
                ),
                # Graphe Réseau
                html.Div(
                    style={"flex": "1", "min-width": "400px"},
                    children=[
                        html.H3("Graphe Réseau"),
                        html.Img(
                            id="network-graph",
                            style={"width": "100%", "border": "1px solid black"},
                        ),
                    ],
                ),
            ],
        ),
    ],
)

# Callbacks pour mettre à jour le tableau et le graphique
@app.callback(
    Output("data-table", "data"),
    [Input("filter-input", "value")],
)
def filter_table(filter_value):
    if not filter_value:
        return df.to_dict("records")
    filtered_df = df[
        df.apply(
            lambda row: row.astype(str).str.contains(filter_value, case=False, na=False).any(),
            axis=1,
        )
    ]
    return filtered_df.to_dict("records")


@app.callback(
    Output("line-chart", "figure"),
    [Input("country-dropdown", "value")],
)
def update_chart(selected_country):
    filtered_df = df[df["country"] == selected_country]
    fig = px.line(
        filtered_df,
        x="year",
        y="pop",
        title=f"Population de {selected_country} dans le temps",
    )
    fig.update_layout(
        plot_bgcolor="#f9f9f9",
        paper_bgcolor="#f4f4f9",
        title={"x": 0.5},
        xaxis_title="Année",
        yaxis_title="Population",
    )
    return fig


# Callback pour le graphe réseau
@app.callback(
    Output("network-graph", "src"),
    [],
)
def generate_network():
    G = nx.Graph()
    my_name = NewData['My Name'].iloc[0]
    G.add_node(my_name)

    for _, row in NewData.iterrows():
        company = row['Company']
        count = row['Count']
        G.add_node(company)
        G.add_edge(my_name, company, weight=count)

    plt.figure(figsize=(8, 5))
    pos = nx.spring_layout(G)
    weights = [G[u][v]['weight'] for u, v in G.edges()]
    edge_width = [w / max(weights) * 5 for w in weights]

    nx.draw(
        G,
        pos,
        with_labels=True,
        node_size=300,
        node_color="skyblue",
        font_size=8,
        edge_color="gray",
        width=edge_width,
    )
    nx.draw_networkx_edge_labels(
        G,
        pos,
        edge_labels={(u, v): G[u][v]['weight'] for u, v in G.edges()},
        font_size=10,
        font_color="red",
    )
    plt.title("Réseau interactif", fontsize=15)
    img_path = "/mnt/data/network_graph.png"
    plt.savefig(img_path)
    plt.close()
    return img_path


# Lancer l'application
if __name__ == "__main__":
    app.run_server(debug=True)
